In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
url = 'https://docs.google.com/spreadsheets/d/1HcdISgCl3s4RpWkJa8m-G1JjfKzd8qf2WY2Xcw32D7U/pub?gid=1371955398&single=true&output=csv'

In [9]:
def full_cast_counterpick(row):
    full_cast_events = [
        "IYL Season 1",
        "IYL Season 2",
        "IYL Season 3",
        "IYL Season 4",
        "IYL Season 5",
        "NFTT Round 1",
        "NFTT Round 2",
        "NFTT Round 3",
        "NFTT Round 4",
        "NFTT Round 5",
        "NFTT Round 6",
        "NFTT Round 7",
        "NFTT Round 8",
        "Summer Smash",
        "Summer Smash II",
        "Summer Smash III",
        "Summer Smash IV",
        "Slippery Spag Saturday I",
        "Slippery Spag Saturday 2",
        "Slippery Spag SUNDAY 3",
        "Slippery Spag Saturday 4",
        "Slippery Spag Sunday 5",
        "Slippery Spag Saturday 6",
        "Slippery Spag Sunday 7",
        "Slippery Spag Sunday 9",
        "Topanda League 1",
        "Topanda League 2",
    ]
    return not any([
        row.char_select_random,
        row.char_select_locked,
        row.event not in full_cast_events
    ]) 

In [10]:
import pandas
import re

historical_record = pandas.read_csv(url)
historical_record['Match Date'] = pandas.to_datetime(historical_record['Match Date'], infer_datetime_format=True)
historical_record.columns = [re.sub('\W+', '_', col.lower()).strip('_') for col in historical_record.columns]
historical_record.format_restricted.replace(to_replace=['.', 'Restricted'], value=[False, True], inplace=True)
historical_record.format_team.replace(to_replace=['.', 'Team'], value=[False, True], inplace=True)
historical_record.char_select_random.replace(to_replace=['.', 'Random'], value=[False, True], inplace=True)
historical_record.char_select_locked.replace(to_replace=['.', 'Locked'], value=[False, True], inplace=True)
historical_record.set_length_non_ft3_ft4.replace(to_replace=['.', 'non-FT3/FT4'], value=[False, True], inplace=True)
historical_record.set_win_1 = historical_record.set_win_1.fillna(0)
historical_record.set_win_2 = historical_record.set_win_2.fillna(0)
historical_record.wins_1 = historical_record.wins_1.fillna(0)
historical_record.wins_2 = historical_record.wins_2.fillna(0)
historical_record['standard_format'] = historical_record.apply(full_cast_counterpick, axis=1)
historical_record.character_1 = historical_record.character_1.apply(lambda n: n.title())
historical_record.character_2 = historical_record.character_2.apply(lambda n: n.title())

Recurrence Relation:
  * `meta_impact(m, n) = 1 + counterpicks_2 * mu_win * meta_impact(m-1, n) + counterpicks_1 * (1 - mu_win) * meta_impact(m, n-1)`
  * `meta_impact(0, n) = 0`
  * `meta_impact(m, 0) = 0`

Counterpicks:

  * Pmn = Probability of match m/n
  * Wmn = Probability of m winning match m/n
  * Cmn = Probability of counterpicking m with n


    Paa = Waa * Caa + Wab * Caa + Wba * 0   + Wbb * 0
    Pab = Waa * Cab + Wab * Cab + Wba * 0   + Wbb * 0
    Pba = Waa * 0   + Wab * 0   + Wba * Cba + Wbb * Cba
    Pbb = Waa * 0   + Wab * 0   + Wba * Cbb + Wbb * Cbb
    [Caa, Caa, 0,   0      [Waa
     Cab, Cab, 0,   0       Wab
     0,   0,   Cba, Cba     Wba
     0,   0,   Cbb, Cbb] *  Wbb]
    
    Paa = (1-Waa) * Caa + (1-Wab) * 0   + (1-Wba) * Caa + (1-Wbb) * 0
    Pab = (1-Waa) * 0   + (1-Wab) * Cba + (1-Wba) * 0   + (1-Wbb) * Cba
    Pba = (1-Waa) * Cab + (1-Wab) * 0   + (1-Wba) * Cab + (1-Wbb) * 0
    Pbb = (1-Waa) * 0   + (1-Wab) * Cbb + (1-Wba) * 0   + (1-Wbb) * Cbb
    [Caa, 0,   Caa, 0          [Waa
     0,   Cba, 0,   Cba         Wab
     Cab, 0,   Cab, 0           Wba
     0,   Cbb, 0,   Cbb] * 1 -  Wbb]

In [11]:
historical_record.event.unique()

array(['IYL Season 1', 'Fast Timer', 'Triple Threat', 'IYL Season 2',
       'Rock Brigade', 'Tag Team League', 'Something Worthwhile',
       'Impromptu Tournament', 'Swiss Tournament', 'Glimpse of the Master',
       'Feature Film Fest', 'Mid Timer', 'Steam Pre-Release',
       'Novice Mentorament', 'Los Angeles Meetup',
       'Emperor Series I - Week 1', 'Double Blind', 'Team Draft',
       'Emperor Series I - Week 2', 'Summer Smash II',
       'Emperor Series I - Week 3', 'IYL Season 3',
       'Emperor Series I - Championship', 'Solo Showdown', 'Bloody Sunday',
       'Emperor Series II - Week 1', 'Emperor Series II - Week 2',
       'Emperor Series II - Week 3', 'Emperor Series II - Championship',
       'Speed of Light', 'Los Angeles Meetup 2', 'White Light White Heat',
       'Topanda League 1', 'Emperor Series III - Week 1',
       'Emperor Series III - Week 2', "Lum's Lucky Lottery",
       'Emperor Series III - Week 3', 'Emperor Series III - Championship',
       'Slippery 

In [12]:
def counterpicks(matches):
    for pick, counterpick in zip(matches, matches[1:] + [None]):
        if counterpick is None:
            if matches[0]['set_win_1']:
                for _ in range(int(pick['wins_1']) - 1):
                    yield (pick['character_1'], pick['character_2'])
                for _ in range(int(pick['wins_2'])):
                    yield (pick['character_2'], pick['character_1'])
            else:
                for _ in range(int(pick['wins_1'])):
                    yield (pick['character_1'], pick['character_2'])
                for _ in range(int(pick['wins_2'] - 1)):
                    yield (pick['character_2'], pick['character_1'])
                
        else:
            if pick['character_1'] != counterpick['character_1']:
                for _ in range(int(pick['wins_1'])):
                    yield (pick['character_1'], pick['character_2'])
                for _ in range(int(pick['wins_2'] - 1)):
                    yield (pick['character_2'], pick['character_1'])
                yield (pick['character_2'], counterpick['character_1'])
            else:
                for _ in range(int(pick['wins_1'] - 1)):
                    yield (pick['character_1'], pick['character_2'])
                for _ in range(int(pick['wins_2'])):
                    yield (pick['character_2'], pick['character_1'])
                yield (pick['character_1'], counterpick['character_2'])

In [13]:
sets = historical_record[historical_record.standard_format].groupby(['match_date', 'event', 'player_1', 'player_2'])

In [14]:
cast = historical_record[historical_record.standard_format].character_1.unique()
matchups = [(l, r) for l in cast for r in cast]

In [15]:
counterpick_counts = pandas.DataFrame(
    data=(
        {'pick': pick, 'counter': counter}
        for (key, set) in sets
        for (pick, counter) in counterpicks([row for (ix, row) in set.iterrows()])
    )
).groupby(['pick', 'counter']).size().reindex(
    pandas.MultiIndex.from_product([cast, cast], names=['pick', 'counter'])
).fillna(0)

In [16]:
blind_picks = pandas.DataFrame(
    {
        'character_1': list(set['character_1'])[0],
        'character_2': list(set['character_2'])[0],
    }
    for (key, set) in sets
)

blind_pick_counts = blind_picks['character_1'].append(blind_picks['character_2']).value_counts()

In [17]:
half_matchup = historical_record.groupby(['character_1', 'character_2']).sum()[['wins_1', 'wins_2']]
reversed_matchup = half_matchup.reset_index().rename(columns={
    'character_1': 'character_2',
    'character_2': 'character_1',
    'wins_1': 'wins_2',
    'wins_2': 'wins_1',
}).set_index(['character_1', 'character_2'])
matchup_pcts = (half_matchup + reversed_matchup).apply(lambda r: r['wins_1']/(r['wins_1'] + r['wins_2']), axis=1).loc[matchups]

In [18]:
identity = pandas.DataFrame({
    current: {
        target: 1 if target == current else 0
        for target in matchups
    } for current in matchups
})

## Computational

In [19]:
# mu_win = pandas.DataFrame(
#     data=matchup_pcts.loc[cast].values,
#     #index=matchup_pcts.reset_index().apply(lambda r: (r.character_1, r.character_2), axis=1),
#     index=pandas.MultiIndex.from_product([cast, cast], names=['character_1', 'character_2']),
# ).loc[matchups]
blind_pcts = blind_pick_counts / blind_pick_counts.sum()
blind_mu_pcts = pandas.Series(
    data=[pct_l*pct_r for pct_l in blind_pcts.values for pct_r in blind_pcts.values],
    index=pandas.MultiIndex.from_product([blind_pcts.index, blind_pcts.index])
)

In [20]:
import itertools

class Meta:
    def __init__(self, exclude=[]):
        self.mu_exclude = exclude + list(itertools.product(cast, exclude))
        self.mu_include = list(set(cast) - set(self.mu_exclude))
        self.blind_mu_pcts = blind_mu_pcts.drop(self.mu_exclude)
        self.blind_mu_pcts /= self.blind_mu_pcts.sum()
        
        self.matchups = [(l, r) for (l, r) in matchups if l not in exclude and r not in exclude]
        
        self.win_count = pandas.DataFrame({
            matchup: {
                c: 1 if c == matchup[0] else 0
                for c in self.mu_include
            } for matchup in self.matchups
        })
        
        self.lose_count = pandas.DataFrame({
            matchup: {
                c: 1 if c == matchup[1] else 0
                for c in self.mu_include
            } for matchup in self.matchups
        })
        
        self.counterpick_pcts = counterpick_counts.drop(self.mu_exclude).groupby('pick').apply(lambda g: g / g.sum())
        self.counterpicks_2 = pandas.DataFrame({
            before: {
                after: 0 if before[0] != after[0] else self.counterpick_pcts.loc[before[0], after[1]]
                for after in self.matchups
            } for before in self.matchups
        })
        
        self.counterpicks_1 = pandas.DataFrame({
            before: {
                after: 0 if before[1] != after[1] else self.counterpick_pcts.loc[before[1], after[0]]
                for after in self.matchups
            } for before in self.matchups
        })
        
        self.matchup_pct_win = pandas.DataFrame({
            current: {
                target: matchup_pcts[current] if current == target else 0
                for target in self.matchups
            } for current in self.matchups
        })
        self.matchup_pct_lose = pandas.DataFrame({
            current: {
                target: 1-matchup_pcts[current] if current == target else 0
                for target in self.matchups
            } for current in self.matchups
        })

        self._meta_influence = {}

    def _matchup_influence(self, remaining_wins_1, remaining_wins_2):
        ix = (remaining_wins_1, remaining_wins_2)
        if ix not in self._meta_influence:
            if remaining_wins_1 == 0:
                self._meta_influence[ix] = self.win_count * 0
            elif remaining_wins_2 == 0:
                self._meta_influence[ix] = self.win_count * 0
            else:
                inf_win = self._matchup_influence(remaining_wins_1 - 1, remaining_wins_2)
                inf_lose = self._matchup_influence(remaining_wins_1, remaining_wins_2 - 1)
                self._meta_influence[ix] = (
                    (inf_win.dot(self.counterpicks_2) + self.win_count).dot(self.matchup_pct_win) +
                    (inf_lose.dot(self.counterpicks_1) + self.lose_count).dot(self.matchup_pct_lose)
                )


        return self._meta_influence[ix]

    def matchup_influence(self, remaining_wins_1, remaining_wins_2):
        return self._matchup_influence(remaining_wins_1, remaining_wins_2).dot(self.blind_mu_pcts)
    
    def character_influence(self, remaining_wins_1, remaining_wins_2):
        mus = self.matchup_influence(remaining_wins_1, remaining_wins_2)
        return pandas.Series({
            c: mus.loc[(mus.index.get_level_values(0) == c) | (mus.index.get_level_values(1) == c)].sum()
            for c in self.mu_include
        })

In [23]:
from numpy.linalg import norm

for c in cast:
    meta = Meta([])
    inf = meta.matchup_influence(4, 4)
    norm_inf = inf / inf.sum()
    print(norm(norm_inf - pandas.Series(data=1/(len(inf)), index=inf.index), ord=2))

    print("Expected wins")
    for char, wins in inf.sort_values(ascending=False).iteritems():
        print(f"1. {char}: {wins:.3}")
    print("Percent wins")
    for char, pct in (norm_inf * 100).sort_values(ascending=False).iteritems():
        print(f"1. {char}: {pct:.3}")

0.0991354073278
Expected wins
1. Zane: 0.583
1. Troq: 0.541
1. Setsuki: 0.438
1. Degrey: 0.436
1. Geiger: 0.408
1. Argagarg: 0.389
1. Grave: 0.349
1. Rook: 0.271
1. Onimaru: 0.265
1. Valerie: 0.25
1. Lum: 0.238
1. Gwen: 0.222
1. Quince: 0.221
1. Bbb: 0.203
1. Vendetta: 0.195
1. Menelker: 0.187
1. Gloria: 0.184
1. Midori: 0.179
1. Persephone: 0.152
1. Jaina: 0.116
Percent wins
1. Zane: 10.0
1. Troq: 9.29
1. Setsuki: 7.51
1. Degrey: 7.48
1. Geiger: 6.99
1. Argagarg: 6.67
1. Grave: 5.99
1. Rook: 4.65
1. Onimaru: 4.55
1. Valerie: 4.3
1. Lum: 4.09
1. Gwen: 3.81
1. Quince: 3.8
1. Bbb: 3.48
1. Vendetta: 3.34
1. Menelker: 3.21
1. Gloria: 3.16
1. Midori: 3.07
1. Persephone: 2.61
1. Jaina: 1.99
0.0991354073278
Expected wins
1. Zane: 0.583
1. Troq: 0.541
1. Setsuki: 0.438
1. Degrey: 0.436
1. Geiger: 0.408
1. Argagarg: 0.389
1. Grave: 0.349
1. Rook: 0.271
1. Onimaru: 0.265
1. Valerie: 0.25
1. Lum: 0.238
1. Gwen: 0.222
1. Quince: 0.221
1. Bbb: 0.203
1. Vendetta: 0.195
1. Menelker: 0.187
1. Gloria: 

0.0991354073278
Expected wins
1. Zane: 0.583
1. Troq: 0.541
1. Setsuki: 0.438
1. Degrey: 0.436
1. Geiger: 0.408
1. Argagarg: 0.389
1. Grave: 0.349
1. Rook: 0.271
1. Onimaru: 0.265
1. Valerie: 0.25
1. Lum: 0.238
1. Gwen: 0.222
1. Quince: 0.221
1. Bbb: 0.203
1. Vendetta: 0.195
1. Menelker: 0.187
1. Gloria: 0.184
1. Midori: 0.179
1. Persephone: 0.152
1. Jaina: 0.116
Percent wins
1. Zane: 10.0
1. Troq: 9.29
1. Setsuki: 7.51
1. Degrey: 7.48
1. Geiger: 6.99
1. Argagarg: 6.67
1. Grave: 5.99
1. Rook: 4.65
1. Onimaru: 4.55
1. Valerie: 4.3
1. Lum: 4.09
1. Gwen: 3.81
1. Quince: 3.8
1. Bbb: 3.48
1. Vendetta: 3.34
1. Menelker: 3.21
1. Gloria: 3.16
1. Midori: 3.07
1. Persephone: 2.61
1. Jaina: 1.99
0.0991354073278
Expected wins
1. Zane: 0.583
1. Troq: 0.541
1. Setsuki: 0.438
1. Degrey: 0.436
1. Geiger: 0.408
1. Argagarg: 0.389
1. Grave: 0.349
1. Rook: 0.271
1. Onimaru: 0.265
1. Valerie: 0.25
1. Lum: 0.238
1. Gwen: 0.222
1. Quince: 0.221
1. Bbb: 0.203
1. Vendetta: 0.195
1. Menelker: 0.187
1. Gloria: 

In [ ]:

for c in cast:
    include = ['Jaina', 'Persephone', 'Gloria', 'Quince', 'Bbb', c]
    exclude = list(set(cast) - set(include))
    print(set(include))
    meta = Meta(exclude)
    inf = meta.matchup_influence(4, 4)
    norm_inf = inf / inf.sum()
    #print(norm(norm_inf - pandas.Series(data=1/(len(inf)), index=inf.index), ord=2))

#     print("Expected wins")
#     for char, wins in inf.sort_values(ascending=False).iteritems():
#         print(f"1. {char}: {wins:.3}")
#     print(inf.sum())
    print(norm_inf.max(), norm_inf.min())
#     print("Percent wins")
#     for char, pct in (norm_inf * 100).sort_values(ascending=False).iteritems():
#         print(f"1. {char}: {pct:.3}")